# Lorentz invariance of Feynman amplitudes

In this notebook we test the Lorentz invariance of Feynman amplitudes. We will consider Compton scattering and Møller scattering amplitudes. For this, we only need `numpy` and `qedtool`.

In [1]:
import numpy as np
import qedtool as qtl

## Compton scattering

First, we define all quantities that define the kinematics:

In [2]:
# Constants
m = qtl.constant("electron mass")
e = qtl.constant("elementary charge")

# Other quantities that define the kinematics
p = 0.2
phi = 0.14
theta = np.pi/2
energy = np.sqrt(p**2 + m**2)

Then, all external and internal 4-momenta are defined:

In [3]:
# Initial 4-momenta
kmu_1 = qtl.FourVector(p, p, 0, 0)
pmu_1 = qtl.FourVector(energy, p, np.pi, 0)

# Final 4-momenta
kmu_2 = qtl.FourVector(p, p, theta, phi)
pmu_2 = qtl.FourVector(energy, p, np.pi - theta, phi + np.pi)

# Internal 4-momenta
qmu_1 = pmu_1 + kmu_1
qmu_2 = pmu_1 - kmu_2

Here we let all external particles be right-handed. Users can change this to consider other polarized scattering.

In [4]:
# External particles
electron_1 = qtl.RealParticle.electron(1, pmu_1, "in")
electron_2 = qtl.RealParticle.electron(1, pmu_2, "out")
photon_1 = qtl.RealParticle.photon(1, kmu_1, "in")
photon_2 = qtl.RealParticle.photon(1, kmu_2, "out")

# Internal particles
fermion_1 = qtl.VirtualParticle.electron(qmu_1)
fermion_2 = qtl.VirtualParticle.electron(qmu_2)

Here we retrieve all polarizations and propagators, with which we construct the total amplitude:

In [5]:
# Electron spinors
u1 = electron_1.polarization.bispinor
u2 = electron_2.polarization.bispinor

# Photon polarization matrices
e1 = -1j * e * qtl.slashed(photon_1.polarization)
e2 = -1j * e * qtl.slashed(photon_2.polarization)

# Propagators
g1 = fermion_1.propagator
g2 = fermion_2.propagator

# Total amplitude
amplitude_compton = u2.dot(e2).dot(g1).dot(e1).dot(u1) + u2.dot(e1).dot(g2).dot(e2).dot(u1)

Now we only boost the particles:

In [6]:
# Boost vector
beta = qtl.ThreeVector(0.9, 1.4, 0.7)

# Boosted external particles
electron_1b = qtl.boost(electron_1, beta)
electron_2b = qtl.boost(electron_2, beta)
photon_1b = qtl.boost(photon_1, beta)
photon_2b = qtl.boost(photon_2, beta)

# Boosted internal particles
fermion_1b = qtl.boost(fermion_1, beta)
fermion_2b = qtl.boost(fermion_2, beta)

Now we compute the amplitude with the boosted polarizations and propagators:

In [7]:
# Boosted polarizations and propagators
u1b = electron_1b.polarization.bispinor
u2b = electron_2b.polarization.bispinor            
e1b = -1j * e * qtl.slashed(photon_1b.polarization)
e2b = -1j * e * qtl.slashed(photon_2b.polarization)
g1b = fermion_1b.propagator
g2b = fermion_2b.propagator
    
# Total amplitude
amplitude_compton_b = u2b.dot(e2b).dot(g1b).dot(e1b).dot(u1b) + u2b.dot(e1b).dot(g2b).dot(e2b).dot(u1b)

Comparing the unboosted and boosted amplitudes:

In [8]:
decimals_compton = 15
print(np.round(amplitude_compton_b, decimals_compton) == np.round(amplitude_compton, decimals_compton))

True


Up to 15 decimals, they are the same, i.e. the amplitude is Lorentz invariant. The photon polarization matrices are rank-2 spin tensors that satisfy

$$\gamma^\mu (\Lambda^{\hphantom{\mu}\nu}_\mu\epsilon_\nu) = \Lambda_{1/2}(\gamma^\mu\epsilon_\mu)\Lambda^{-1}_{1/2}\,.$$

We verify this relation with the following lines:

In [9]:
spin_boost = qtl.boost_matrix(beta, "bispinor")
spin_boost_inv = np.linalg.inv(spin_boost)
print(np.round(e1b, 15) == np.round(spin_boost.dot(e1).dot(spin_boost_inv), 15))

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]]


## Møller scattering

Here we will compute and boost the scattering amplitude for elastic electron-electron scattering, i.e. Møller scattering. Here we define the 4-momenta from the CM frame:

In [10]:
# Initial 4-momenta
pmu_1i = qtl.FourVector(energy, p, 0, 0)
pmu_2i = qtl.FourVector(energy, p, np.pi, 0)

# Final 4-momenta
pmu_1f = qtl.FourVector(energy, p, theta, phi)
pmu_2f = qtl.FourVector(energy, p, np.pi - theta, phi + np.pi)

# Internal 4-momenta
qmu_t = pmu_1i - pmu_1f
qmu_u = pmu_1i - pmu_2f

Then we define the initial and final electrons:

In [11]:
# External electrons
electron_1i = qtl.RealParticle.electron(1, pmu_1i, "in")
electron_2i = qtl.RealParticle.electron(1, pmu_2i, "in")
electron_1f = qtl.RealParticle.electron(1, pmu_1f, "out")
electron_2f = qtl.RealParticle.electron(1, pmu_2f, "out")

The scattering amplitudes of the $t$- and $u$-channels can be written as $\smash{i\mathcal{M} \propto \frac{1}{q^2}\tilde{\imath}_\mu\tilde{\jmath}^\mu}$ where $q^2$ is the Lorentzian norm of the virtual photon's 4-momentum, and $\tilde{\imath}$ and $\tilde{\jmath}$ are U(1) currents of the form $\bar{u}\gamma^\mu u$. Therefore we retrieve polarizations and define the currents:

In [12]:
# Polarizations
u1i = electron_1i.polarization
u2i = electron_2i.polarization
u1f = electron_1f.polarization
u2f = electron_2f.polarization

# Dirac currents
jmu_11 = qtl.FourVector.dirac_current(u1f, u1i)
jmu_12 = qtl.FourVector.dirac_current(u2f, u1i)
jmu_21 = qtl.FourVector.dirac_current(u1f, u2i)
jmu_22 = qtl.FourVector.dirac_current(u2f, u2i)

We calculate the total amplitude as the difference between the $t$-channel and $u$-channel amplitudes, as an exchange of electrons yields a minus sign:

In [13]:
amplitude_moller = jmu_11 * jmu_22 / (qmu_t * qmu_t) - jmu_21 * jmu_12  / (qmu_u * qmu_u)

By boosting the external electrons, we boost their spinors: $u \to \Lambda_{\frac{1}{2}}u$ and $\bar{u} \to \bar{u}\Lambda^{-1}_{\frac{1}{2}}$:

In [14]:
# Boosted external electrons
electron_1ib = qtl.boost(electron_1i, beta)
electron_2ib = qtl.boost(electron_2i, beta)
electron_1fb = qtl.boost(electron_1f, beta)
electron_2fb = qtl.boost(electron_2f, beta)

Retrieving the boosted polarizations to form boosted Dirac currents:

In [15]:
# Boosted internal momenta
qmu_tb = qtl.boost(qmu_t, beta)
qmu_ub = qtl.boost(qmu_u, beta)

# Boosted polarizations
u1ib = electron_1ib.polarization
u2ib = electron_2ib.polarization
u1fb = electron_1fb.polarization
u2fb = electron_2fb.polarization

# Boosted Dirac currents
jmu_11b = qtl.FourVector.dirac_current(u1fb, u1ib)
jmu_12b = qtl.FourVector.dirac_current(u2fb, u1ib)
jmu_21b = qtl.FourVector.dirac_current(u1fb, u2ib)
jmu_22b = qtl.FourVector.dirac_current(u2fb, u2ib)

Here we compute the boosted amplitude:

In [16]:
amplitude_moller_b = jmu_11b * jmu_22b / (qmu_tb * qmu_tb) - jmu_21b * jmu_12b  / (qmu_ub * qmu_ub)

We round both amplitudes down to some decimal, since floating point errors are inevitable. Comparing the two:

In [17]:
decimals_moller = 13
print(np.round(amplitude_moller_b, decimals_moller) == np.round(amplitude_moller, decimals_moller))

True


The unboosted and boosted amplitudes are equal up to 13 decimals. We should also find that

$$\Lambda^{\mu}_{\hphantom{\mu}\nu}\,\tilde{\jmath}^\nu = \bar{u}\Lambda^{-1}_{1/2}\gamma^\mu\Lambda_{1/2}u\,,$$

which we test with the following lines:

In [18]:
# 4-vector boost of jmu_11
jmu_11b2 = qtl.boost(jmu_11, beta)

# Comparison of the two (test of the equation above)
print(np.round(jmu_11b2.vector, 14) == np.round(jmu_11b.vector, 14))

[ True  True  True  True]
